In [125]:
import warnings
warnings.filterwarnings('ignore')
import bioframe as bf
from gpn.data import load_table, Genome
import numpy as np
import polars as pl
from scipy.special import softmax
from tqdm import tqdm

In [126]:
NUCLEOTIDES = list("ACGT")
chrom = "22"
model = "multiz100way/89/128/64/True/defined.phastCons.percentile-75_0.05_0.001/medium/0.1/42/30000/True/True/True"

In [127]:
anc_path = "../../results/ancestral/homo_sapiens_ancestor_GRCh38/homo_sapiens_ancestor_22.fa"

In [128]:
anc_seq = Genome(anc_path)._genome["ANCESTOR_for_chromosome:GRCh38:22:1:50818468:1"].upper()
anc_seq = np.frombuffer(anc_seq.encode("ascii"), dtype="S1")
anc_seq

array([b'.', b'.', b'.', ..., b'.', b'.', b'.'], dtype='|S1')

In [120]:
# https://ftp.ensembl.org/pub/release-109/fasta/ancestral_alleles/

In [31]:
annotation = load_table("../../results/annotation.gtf.gz")[["chrom", "start", "end", "feature"]]
annotation = annotation[annotation.chrom==chrom]
annotation

,chrom,start,end,feature
3261452,22,11827522,11910358,gene
3261453,22,11827522,11910358,transcript
3261454,22,11827522,11827658,exon
3261455,22,11832760,11832915,exon
3261456,22,11867114,11867145,exon
...,...,...,...,...
3332625,22,27750775,27750914,three_prime_utr
3332626,22,27750063,27750228,three_prime_utr
3332627,22,27750677,27791883,transcript
3332628,22,27791746,27791883,exon


In [108]:
annotation = annotation.replace({
    "five_prime_utr": "5' UTR",
    "three_prime_utr": "3' UTR",
})

In [109]:
annotation.feature.value_counts()

feature
exon              34347
CDS               18387
transcript         5351
3' UTR             4148
5' UTR             3538
start_codon        2098
stop_codon         1885
gene               1412
Selenocysteine       12
Name: count, dtype: int64

In [110]:
features = [
    "CDS",
    "5' UTR",
    "3' UTR",
]

feature_intervals = {
    feature: bf.merge(annotation[annotation.feature==feature]).drop(columns="n_intervals") 
    for feature in features
}

{'CDS':      chrom     start       end
 0       22  15528191  15529136
 1       22  15690077  15690709
 2       22  15695370  15695485
 3       22  15695644  15695818
 4       22  15697373  15697529
 ...    ...       ...       ...
 4522    22  50777951  50777981
 4523    22  50779054  50779121
 4524    22  50780180  50780226
 4525    22  50780708  50780718
 4526    22  50782187  50782294
 
 [4527 rows x 3 columns],
 "5' UTR":      chrom     start       end
 0       22  15690025  15690077
 1       22  16592550  16592810
 2       22  16808073  16808083
 3       22  16821606  16821699
 4       22  16825290  16825411
 ...    ...       ...       ...
 1264    22  50729571  50729660
 1265    22  50738203  50738235
 1266    22  50782294  50782351
 1267    22  50782768  50783286
 1268    22  50783390  50783667
 
 [1269 rows x 3 columns],
 "3' UTR":      chrom     start       end
 0       22  15697532  15697629
 1       22  15698661  15698768
 2       22  15700077  15700215
 3       22  15702685

In [148]:
path = f"../../results/positions/{chrom}/processed_logits/{model}.parquet"
V = (
    pl.read_parquet(path)
)
V = V.with_columns(anc=anc_seq[V["pos"]-1])
V = V.with_columns(anc=pl.col("anc").cast(str))
V = V.with_columns(
    pl.DataFrame(softmax(V.select(NUCLEOTIDES), axis=1), schema=NUCLEOTIDES)
)
for c in ["ref", "anc"]:
    V = (
        V.with_columns(
            pl.when(pl.col(c) == "A").then(pl.col("A"))
            .when(pl.col(c) == "C").then(pl.col("C"))
            .when(pl.col(c) == "G").then(pl.col("G"))
            .when(pl.col(c) == "T").then(pl.col("T"))
            .alias(f"prob_{c}")
        )
        .with_columns(
            (1 / pl.col(f"prob_{c}")).alias(f"perplexity_{c}"),
            (pl.col(f"prob_{c}") == pl.max_horizontal(NUCLEOTIDES))
            .alias(f"accuracy_{c}").cast(float)
        )
    )
intervals = (
    V.with_columns(
        (pl.col("pos") - 1).alias("start"),
        pl.col("pos").alias("end")
    )
    .select(["chrom", "start", "end"])
    .to_pandas()
)
for f, x in tqdm(feature_intervals.items()):
    V = V.with_columns(
        pl.from_pandas(bf.count_overlaps(intervals, x)["count"] > 0)
        .alias(f)
    )
V

100%|██████████| 3/3 [03:28<00:00, 69.62s/it]


chrom,pos,ref,A,C,G,T,anc,prob_ref,perplexity_ref,accuracy_ref,prob_anc,perplexity_anc,accuracy_anc,CDS,5' UTR,3' UTR
str,i64,str,f32,f32,f32,f32,str,f32,f32,f64,f32,f32,f64,bool,bool,bool
"""22""",10510065,"""A""",0.677927,0.101803,0.118962,0.101308,""".""",0.677927,1.475085,1.0,null,null,null,false,false,false
"""22""",10510066,"""A""",0.678113,0.104195,0.11466,0.103032,""".""",0.678113,1.47468,1.0,null,null,null,false,false,false
"""22""",10510067,"""T""",0.107364,0.133747,0.101849,0.657041,""".""",0.657041,1.521976,1.0,null,null,null,false,false,false
"""22""",10510068,"""A""",0.680645,0.096985,0.123955,0.098416,""".""",0.680645,1.469195,1.0,null,null,null,false,false,false
"""22""",10510069,"""T""",0.120914,0.108757,0.121446,0.648883,""".""",0.648883,1.54111,1.0,null,null,null,false,false,false
"""22""",10510070,"""G""",0.541647,0.093849,0.229348,0.135156,""".""",0.229348,4.360189,0.0,null,null,null,false,false,false
"""22""",10510071,"""T""",0.11857,0.059215,0.754159,0.068056,""".""",0.068056,14.693797,0.0,null,null,null,false,false,false
"""22""",10510072,"""A""",0.721582,0.080565,0.113504,0.084349,""".""",0.721582,1.385843,1.0,null,null,null,false,false,false
"""22""",10510073,"""T""",0.084662,0.097041,0.083532,0.734765,""".""",0.734765,1.36098,1.0,null,null,null,false,false,false


In [149]:
V = V.with_columns(
    pl.when(pl.col("CDS")).then(pl.lit("CDS"))
    .when(pl.col("5' UTR")).then(pl.lit("5' UTR"))
    .when(pl.col("3' UTR")).then(pl.lit("3' UTR"))
    .otherwise(pl.lit("Non-exonic"))
    .alias("Region")
)
V["Region"].value_counts()

Region,count
str,u32
"""3' UTR""",785066
"""5' UTR""",189036
"""Non-exonic""",37423927
"""CDS""",755604


In [150]:
V.group_by("Region").agg(
    pl.mean("perplexity_ref").alias("mean perplexity"),
    pl.median("perplexity_ref").alias("median perplexity"),
    pl.mean("accuracy_ref").alias("mean accuracy"),
).sort("mean accuracy", descending=True)

Region,mean perplexity,median perplexity,mean accuracy
str,f32,f32,f64
"""CDS""",4.877965,1.032106,0.915296
"""5' UTR""",3.203826,1.645802,0.836999
"""3' UTR""",3.116771,1.687933,0.821391
"""Non-exonic""",2.528364,1.977608,0.690292


In [151]:
V_diff = V.filter(
    pl.col("anc").is_in(NUCLEOTIDES),
    pl.col("anc") != pl.col("ref")
)
V_diff

chrom,pos,ref,A,C,G,T,anc,prob_ref,perplexity_ref,accuracy_ref,prob_anc,perplexity_anc,accuracy_anc,CDS,5' UTR,3' UTR,Region
str,i64,str,f32,f32,f32,f32,str,f32,f32,f64,f32,f32,f64,bool,bool,bool,str
"""22""",10518459,"""A""",0.008285,0.008497,0.011397,0.971821,"""G""",0.008285,120.705559,0.0,0.011397,87.740578,0.0,false,false,false,"""Non-exonic"""
"""22""",10518501,"""T""",0.134272,0.10653,0.658977,0.100222,"""G""",0.100222,9.977855,0.0,0.658977,1.517505,1.0,false,false,false,"""Non-exonic"""
"""22""",10519374,"""A""",0.835341,0.016677,0.131142,0.016841,"""T""",0.835341,1.197116,1.0,0.016841,59.380619,0.0,false,false,false,"""Non-exonic"""
"""22""",10519601,"""T""",0.034935,0.048267,0.035382,0.881416,"""C""",0.881416,1.134538,1.0,0.048267,20.718164,0.0,false,false,false,"""Non-exonic"""
"""22""",10520484,"""A""",0.166912,0.333243,0.335201,0.164645,"""G""",0.166912,5.991196,0.0,0.335201,2.983286,1.0,false,false,false,"""Non-exonic"""
"""22""",10520648,"""T""",0.223243,0.429472,0.070539,0.276747,"""C""",0.276747,3.61341,0.0,0.429472,2.328441,1.0,false,false,false,"""Non-exonic"""
"""22""",10521092,"""T""",0.225548,0.222702,0.243162,0.308588,"""G""",0.308588,3.240564,1.0,0.243162,4.112489,0.0,false,false,false,"""Non-exonic"""
"""22""",10521902,"""A""",0.966232,0.006049,0.022035,0.005684,"""G""",0.966232,1.034949,1.0,0.022035,45.381908,0.0,false,false,false,"""Non-exonic"""
"""22""",10521936,"""A""",0.128062,0.406185,0.15614,0.309613,"""C""",0.128062,7.80874,0.0,0.406185,2.461933,1.0,false,false,false,"""Non-exonic"""


In [153]:
V_diff.group_by("Region").agg(
    #pl.mean("perplexity_ref").alias("mean perplexity ref"),
    #pl.median("perplexity_ref").alias("median perplexity ref"),
    pl.mean("accuracy_ref").alias("mean accuracy ref"),
    #pl.mean("perplexity_anc").alias("mean perplexity anc"),
    #pl.median("perplexity_anc").alias("median perplexity anc"),
    pl.mean("accuracy_anc").alias("mean accuracy anc"),
).sort("mean accuracy anc", descending=True)

Region,mean accuracy ref,mean accuracy anc
str,f64,f64
"""CDS""",0.227335,0.679959
"""5' UTR""",0.177312,0.678742
"""3' UTR""",0.245011,0.60324
"""Non-exonic""",0.268406,0.515225
